In [16]:
%matplotlib inline
import warnings
from collections import Counter, OrderedDict
from pathlib import Path

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from scipy import sparse
from scipy.spatial.distance import pdist, squareform

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, ScalarFormatter
import seaborn as sns

# spacy, textblob and nltk for language processing
from textblob import TextBlob, Word

# sklearn for feature extraction & modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix
from sklearn.externals import joblib

import lightgbm as lgb

import json
from time import clock, time

from sklearn.pipeline import Pipeline


In [3]:
sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

## Topic Modeling BBC news

In [4]:
DATA_DIR = Path('..', 'data')
path = DATA_DIR / 'bbc'
files = sorted(list(path.glob('**/*.txt')))
doc_list = []
for i, file in enumerate(files):
    topic = file.parts[-2]
    article = file.read_text(encoding='latin1').split('\n')
    heading = article[0].strip()
    body = ' '.join([l.strip() for l in article[1:]])
    doc_list.append([topic, heading, body])
docs = pd.DataFrame(doc_list, columns=['topic', 'heading', 'body'])
docs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    2226 non-null   object
 1   heading  2226 non-null   object
 2   body     2226 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [12]:
y = pd.factorize(docs.topic)[0]
X = docs.body
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
def model_Pipeline():
    pipeline=Pipeline([
#         ('tfidf', TfidfVectorizer(max_df=.95,min_df=2,stop_words="english")),
                ('tfidf', CountVectorizer(stop_words="english")),
#                         ('tfidf', TfidfVectorizer()),

                     ('clf', MultinomialNB()),
#                      ('clf', GradientBoostingClassifier()),
#                      ('clf', RandomForestClassifier())
#                      ('clf', CatBoostClassifier())
#                      ('clf', xgb())
#                         ('clf', LogisticRegression())
#                        ('clf', MultinomialNB())
                    ])
    return pipeline

model=model_Pipeline()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [36]:
accuracy_score(y_test, predictions)

0.9820466786355476

In [37]:
pd.DataFrame(confusion_matrix(y_test,predictions))

,0,1,2,3,4
0,114,0,1,0,1
1,1,100,2,0,1
2,1,0,111,0,1
3,0,0,1,115,0
4,0,0,1,0,107


## Yelp sentiment